In [57]:
import numpy as np
import tensorflow as tf
import os
tf.compat.v1.disable_eager_execution()
# from keras.preprocessing.image import image
from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input #-  when not using resnet 18
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.models import Sequential,Model,load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D,GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import SGD, Adam
import cv2
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
import pickle
from random import randint, shuffle
from classification_models import Classifiers
import keras
from tensorflow.python.client import device_lib


In [58]:
import keras
print(tf.__version__)
EPOCHS = 2
BS = 8
INIT_LR = 1e-5
le = LabelEncoder()
classification_algo = False
if classification_algo:
    ResNeXt50, preprocess_input = Classifiers.get('resnext50')

2.3.1


In [59]:
imagePaths = list(paths.list_images('fight_image_dataset'))

In [60]:
def while_loop_image(half_batch_size, label):
    j = 0
    list_required = []
    while j < half_batch_size:
        img_num = randint(0, 20000)
        image = load_img(f'fight_image_dataset/{label}/img{img_num}.jpg', target_size=(224,224))
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = preprocess_input(image)
        list_required.append((label, image))
        j +=1
    return list_required

In [63]:
base_model = None
image_getter = 0
model_list = []
while image_getter < 40000:
    data = []
    labels = []
    i = 0
    
    while i < 16:
        fight_list = while_loop_image(BS/2, 'fight') 
        non_fight_list = while_loop_image(BS/2, 'no_fight')
        real_list = fight_list + non_fight_list
        shuffle(real_list)
        labels_mini, data_mini = zip(*real_list)
        data += list(data_mini)
        labels += list(labels_mini)
        i+=BS
    
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    labels = to_categorical(labels, 2)
    
    data = np.array(data)
    data = np.squeeze(data)
    labels = np.array(labels)
    print(labels.shape)
    labels = np.squeeze(labels)
    (trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)
    
    img_height, img_width, channel = 224, 224, 3
    num_classes = 2
    if base_model is None:
        #print("this is printed in if statement")
        base_model = ResNet50(weights= None, include_top=False, input_shape= (img_height,img_width,3))
    else:
        print("This should not be printing for 1st time")
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    preds = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs= base_model.input, outputs=preds)
    print(trainY.shape)
    adam = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
    model.compile(optimizer= adam, loss='binary_crossentropy', metrics=['accuracy'])
    H = model.fit(trainX, trainY, batch_size=BS, epochs = EPOCHS, validation_data=(testX, testY))
    model_list.append()
    image_getter += 20000


(16, 2)
(12, 2)
Train on 12 samples, validate on 4 samples
Epoch 1/2


KeyboardInterrupt: 

In [ ]:
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
preds = Dense(num_classes, activation='softmax')(x)
model = Model(inputs= base_model.input, outputs=preds)

In [ ]:
    image_getter += 20000
    base_model = model
    del model

In [13]:
labels = le.fit_transform(labels)
